In [ ]:
# This is a demo script showing how to use the functions
# directly from the compiled Rust core module.

# We assume the compiled Rust library is available as `dx_core`
import dx_core 
import numpy as np

In [ ]:
# 1. DEFINE the dynamical system (e.g., Lorenz Attractor)
# The user provides a standard Python function.
def lorenz_system(t, state):
    # Parameters can be defined inside or passed as arguments
    sigma = 10.0
    rho = 28.0
    beta = 8.0 / 3.0
    
    x, y, z = state
    dx_dt = sigma * (y - x)
    dy_dt = x * (rho - z) - y
    dz_dt = x * y - beta * z
    return np.array([dx_dt, dy_dt, dz_dt])

In [ ]:
# 2. CONFIGURE and SIMULATE the system
# We now call the Rust function directly.
initial_state = np.array([1.0, 1.0, 1.0])
t_start = 0.0
t_end = 50.0
h_initial = 0.01 # The adaptive solver will change this

print("🚀 Running adaptive RK45 simulation...")
# --- Call the new adaptive solver directly ---
# The function now takes tolerance arguments and returns a (trajectory, times) tuple.
trajectory, times = dx_core.solve_rk45_adaptive(
    lorenz_system,
    initial_state,
    t_start=t_start,
    t_end=t_end,
    h=h_initial,
    abstol=1e-8,
    reltol=1e-8
)
print(f"✅ Simulation complete. Generated {trajectory.shape[0]} points.")

In [ ]:
# 3. ANALYZE the results
# We call the analysis functions from our Rust core directly.

# --- Call the new Lyapunov spectrum function ---
print("🧠 Computing Lyapunov spectrum...")
# This function requires more parameters to control the calculation.
# It returns the final spectrum and the history of its convergence.
lyapunov_spectrum, lyapunov_history = dx_core.compute_lyapunov_spectrum(
    lorenz_system,
    initial_state,
    t_transient=10.0,   # Time to let the system settle on the attractor
    t_total=1000.0,     # Total time for the averaging process
    t_reorth=1.0,       # Time between QR re-orthogonalizations
    h_init=h_initial,
    abstol=1e-8,
    reltol=1e-8
)
print(f"✅ Lyapunov spectrum computed: {np.round(lyapunov_spectrum, 4)}")

# --- CHANGE: Call the entropy and stats functions directly ---
print("📊 Computing entropy and statistical measures...")
# We'll analyze the x-component of the trajectory
x_series = trajectory[:, 0]

# Permutation Entropy
perm_entropy = dx_core.compute_permutation_entropy(x_series, m=3, tau=1)
print(f"✅ Permutation Entropy (m=3, tau=1): {perm_entropy:.4f}")

# Approximate Entropy
approx_entropy = dx_core.compute_approximate_entropy(x_series, m=2, r=0.2 * np.std(x_series))
print(f"✅ Approximate Entropy (m=2, r=0.2*std): {approx_entropy:.4f}")

# Invariant Measure (2D Histogram of the x-z plane)
invariant_measure_hist = dx_core.compute_invariant_measure(trajectory[:, [0, 2]], epsilon=0.5)
print(f"✅ Invariant measure computed. Found {len(invariant_measure_hist)} populated bins.")

# Estimate KS-Entropy from the positive Lyapunov exponents
ks_entropy_est = np.sum([le for le in lyapunov_spectrum if le > 0])
print(f"📈 Estimated KS-Entropy from spectrum: {ks_entropy_est:.4f}")

In [ ]:
# 4. VISUALIZE (Conceptual)
# The visualization code would use the variables computed above.
# For example:
# fig1 = dx.visualize.plot_phase_portrait(trajectory, dims=['x', 'y', 'z'])
# fig2 = dx.visualize.plot_lyapunov_convergence(lyapunov_history)
# fig3 = dx.visualize.plot_invariant_measure(invariant_measure_hist) # This would need more work to plot
# fig1.show()